In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import date
import datetime as dt
import re
import pathlib
import os 



cwd = os.getcwd()
pathlib.Path(cwd + "/Articles").mkdir(parents=True, exist_ok=True)
temp = dt.date(1899, 12, 30) # Note, not 31st Dec but 30th!
today = date.today()
store = []


def main():
    days = all_dates()
    for day in reversed(days):
        timestamp = get_date(day)
        link = get_link(day,timestamp)
        soup = ret(link)
        articles = soup.find_all('ul', {'class':'content'})
        print(""+day.strftime('%Y,%m,%d')+"")
        for art in articles:
            art_links = art.find_all('a', href = True)
            for art_link in art_links:
                art_link = 'https://economictimes.indiatimes.com' + art_link['href']
                sub(art_link)
        
        
        
       
def sub(url):
    for x in range(3):
        if(url in store):
            break;
        soup1 = ret(url)
        try:
            headline = soup1.find('h1', {'class':'artTitle font_faus'}).get_text()
            auth = soup1.find('div', {'class':['auth','eventDone']})
            if(not auth):
                auth = soup1.find('span',{'class':'ag'})
            if(not auth):
                auth = soup1.find('div',{'class':'p_auth_list tac'})
            if(not auth):
                auth = soup1.find('div',{'class':'ag tac'}) 
                
            article = soup1.find('div', {'class':'artText'})
            if(not article):
                article = soup1.find('div', {'class':'content1 clearfix'})
                
            dop = re.sub('[^a-zA-Z0-9]', '', soup1.find('time').get_text()).replace('LastUpdated','').replace('Published','')
            dop = dop[:-9]
            dop = dt.datetime.strptime(dop, '%b%d%Y').strftime('%Y-%m-%d')
            
            data = "Source: Economic Times - India;"
            data += "Link: " + url + ";"
            data += "DateOfExtraction: " + today.strftime("%Y-%m-%d") + ";"
            data += "DateOfPublication: " + dop +";"
            data += "Author: " + auth.get_text() +";"
            data += "Title: " + headline +";"
            data += "Article: " + article.get_text() + ";"
            f=open(cwd + "/Articles/" + re.sub('[^a-zA-Z0-9]', '_', headline)+'.txt','w',errors="ignore")
            print(data, file =f)
            f.close()
            store.append(url)
        except:
            time.sleep(1)
            if(x==2):
                print("Irregular data : ", url)
        
        
        
def get_link(day,timestamp):
    link = '/archivelist/year-'+ day.strftime('%Y') +',month-'+ day.strftime('%m') + ',starttime-' + str(int(timestamp)) +'.cms'
    return 'https://economictimes.indiatimes.com' + link
    
def all_dates():
    d1 = dt.date(2002, 1, 1)
    d2 = dt.date(int(today.strftime("%Y")),int(today.strftime("%m")),int(today.strftime("%d")))
    days = [d1 + dt.timedelta(days=x) for x in range((d2-d1).days + 1)]
    return days
    
    
def get_date(date1):    
    delta = date1 - temp
    return float(delta.days) + (float(delta.seconds) / 86400)


def ret(url):
    with requests.Session() as req:
        try:
            r = req.get(url)
        except requests.exceptions.ConnectionError:
            r.status_code = "Connection refused"
        html_response = r.text
        return BeautifulSoup(html_response,'html.parser')

main()